In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep
import utils.optimization as opt

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
dset_idx = 0

In [4]:
seed = 123
rand = np.random.RandomState()

# Load and filter survival data

In [5]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["figo_stage", "age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["vital_status", "survival_time"]
cat_cols = ["race", "ethnicity", "figo_chr"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/survival_data.tsv", event_code)

In [6]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="c")
        .query("vital_status == 1")
        .drop(["vital_status"], axis=1)
        .pipe(pd.get_dummies, columns=cat_cols)
        .reset_index(drop = True)
)
filtered_survival_df.columns = filtered_survival_df.columns.str.replace(' ', '_')

print(filtered_survival_df.shape)

(66, 16)


# Load normalized matrisome count data

In [7]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)

(66, 1009)


# Join survival and count data

In [8]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
)
print(joined_df.shape)

(66, 1023)


In [9]:
filtered_joined_df = prep.filter_outliers_IQR(joined_df, "survival_time", coef=1.5)

# Optimize model

In [10]:
rand.seed(seed)
x_df, y_df = prep.shuffle_data(filtered_joined_df, rand)

In [11]:
mean_baseline = mean_squared_error(y_df.values, np.repeat(np.mean(y_df.values.squeeze()), y_df.shape[0]))
median_baseline = mean_absolute_error(y_df.values, np.repeat(np.median(y_df.values.squeeze()), y_df.shape[0]))
r2_baseline = r2_score(y_df.values, np.repeat(np.mean(y_df.values.squeeze()), y_df.shape[0]))
expl_var_baseline = explained_variance_score(y_df.values, np.repeat(np.mean(y_df.values.squeeze()), y_df.shape[0]))

print(f"L2 baseline: {mean_baseline}")
print(f"L1 baseline: {median_baseline}")
print(f"R2 baseline: {r2_baseline}")
print(f"explained variance baseline: {expl_var_baseline}")

L2 baseline: 216346.93791991402
L1 baseline: 353.1803278688525
R2 baseline: 0.0
explained variance baseline: 0.0


In [12]:
def objective(h_params, X, y, scoring_default, r, verbose=True):
    if verbose:
        print(h_params)
    model = RandomForestRegressor(
        n_estimators=h_params[0],
        max_depth=h_params[1],
        max_features=h_params[2],
        min_samples_split=h_params[3],
        min_samples_leaf=h_params[4],
        bootstrap=h_params[5],
        n_jobs=-1,
        random_state=r
    )
    return -np.mean(cross_val_score(
        model,
        X,
        y,
        cv=KFold(n_splits=5),
        n_jobs=-1,
        scoring=scoring_default
    ))

In [13]:
space = [
    Integer(int(1e2), int(1e3), name="n_estimators"),
    Integer(int(10), int(100), name="max_depth"),
    Categorical(["auto", "sqrt", "log2"], name="max_features"),
    Integer(int(2), int(4), name="min_samples_split"),
    Integer(int(1), int(3), name="min_samples_leaf"),
    Categorical([True, False], name="bootstrap")
]
n_initial = 10 * len(space)
n_calls = 50 * len(space)

In [14]:
scoring_default = "explained_variance"
callback_file = f"{unified_dsets[dset_idx]}_opt_rfr_h_params_{scoring_default}_outliers_removed.tsv"

try:
    os.remove(callback_file)
except OSError:
    pass


res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df, scoring_default, rand),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls = n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, callback_file, n = 5, sep="\t")
)

Iteration No: 1 started. Evaluating function at random point.
[662, 20, 'log2', 3, 3, False]
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 2.7284
Function value obtained: -0.1594
Current minimum: -0.1594
Iteration No: 2 started. Evaluating function at random point.
[535, 99, 'log2', 3, 1, True]
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 2.0263
Function value obtained: -0.1179
Current minimum: -0.1594
Iteration No: 3 started. Evaluating function at random point.
[643, 59, 'log2', 3, 2, True]
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 2.7225
Function value obtained: -0.1121
Current minimum: -0.1594
Iteration No: 4 started. Evaluating function at random point.
[888, 56, 'log2', 3, 2, True]
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 2.8356
Function value obtained: -0.1109
Current minimum: -0.1594
Iteration No: 5 started. Evaluating function at random point.
[858, 17, 'sqrt', 2, 1, True]
Iterati

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 65 ended. Search finished for the next optimal point.
Time taken: 3.1205
Function value obtained: -0.1724
Current minimum: -0.1842
Iteration No: 66 started. Searching for the next optimal point.
[1000, 100, 'sqrt', 4, 3, False]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 66 ended. Search finished for the next optimal point.
Time taken: 3.0044
Function value obtained: -0.1782
Current minimum: -0.1842
Iteration No: 67 started. Searching for the next optimal point.
[122, 100, 'sqrt', 4, 2, False]
Iteration No: 67 ended. Search finished for the next optimal point.
Time taken: 1.3777
Function value obtained: -0.1133
Current minimum: -0.1842
Iteration No: 68 started. Searching for the next optimal point.
[1000, 63, 'sqrt', 2, 3, False]
Iteration No: 68 ended. Search finished for the next optimal point.
Time taken: 3.5595
Function value obtained: -0.1846
Current minimum: -0.1846
Iteration No: 69 started. Searching for the next optimal point.
[1000, 96, 'sqrt', 2, 3, False]
Iteration No: 69 ended. Search finished for the next optimal point.
Time taken: 2.8542
Function value obtained: -0.1738
Current minimum: -0.1846
Iteration No: 70 started. Searching for the next optimal point.
[1000, 99, 'sqrt', 2, 3, False]
Iteration No: 70 ended. Search finis

...

Iteration No: 298 ended. Search finished for the next optimal point.
Time taken: 9.9243
Function value obtained: -0.1540
Current minimum: -0.2212
Iteration No: 299 started. Searching for the next optimal point.
[1000, 10, 'log2', 2, 3, False]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 299 ended. Search finished for the next optimal point.
Time taken: 10.5908
Function value obtained: -0.1629
Current minimum: -0.2212
Iteration No: 300 started. Searching for the next optimal point.
[1000, 10, 'log2', 2, 3, False]


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 300 ended. Search finished for the next optimal point.
Time taken: 9.9777
Function value obtained: -0.1699
Current minimum: -0.2212
